In [77]:
import pandas as pd
#working on integrating this information with mtgjson, but until that feature works, we don't need this.
#import utility


In [78]:
#path to our data, feel free to change this to whatever dataset you want.
pricingPath = 'FAB Price Data.csv'

#Fees for buying and selling. Loosely based on TCGplayer numbers.
#Will be a little more for cheaper cards, and a little less for more expensive cards
sellFee = .12
buyFee = .10

#We don't have a buylist to do a direct comparison to, so we set percentage rates. Used in cell 8
#TODO: move some cells to markdown so we can link to them.
optimisticBuy = .5
pessimisticBuy = .8

#for the simple math we've got set up, this makes it easier down below.
sellCut = 1 - sellFee
buyCut = 1 - buyFee

#If I can't buy a cheeseburger with something, it doesn't actually have value.
cheeseburger = 5

In [79]:
pricingData = pd.read_csv(pricingPath)

In [80]:
#drop the utility columns that we don't care about.
result = pricingData.drop(['Title', 'TCG Direct Low', 'Add to Quantity', 'TCG Marketplace Price', 'My Store Reserve Quantity', 'My Store Price', 'Photo URL', 
'Total Quantity', 'Add to Quantity', 'TCG Marketplace Price', 'My Store Reserve Quantity', 'My Store Price', 'Number'], axis = 1)

In [81]:
result.head()

,TCGplayer Id,Product Line,Set Name,Product Name,Rarity,Condition,TCG Market Price,TCG Low Price With Shipping,TCG Low Price
0,4621888,Flesh & Blood TCG,Arcane Rising,Absorb in Aether (Blue),Rare,Near Mint 1st Edition Normal,0.87,1.24,0.25
1,4621889,Flesh & Blood TCG,Arcane Rising,Absorb in Aether (Blue),Rare,Lightly Played 1st Edition Normal,NaN,1.29,0.30
2,4621890,Flesh & Blood TCG,Arcane Rising,Absorb in Aether (Blue),Rare,Moderately Played 1st Edition Normal,NaN,NaN,NaN
3,4621891,Flesh & Blood TCG,Arcane Rising,Absorb in Aether (Blue),Rare,Heavily Played 1st Edition Normal,NaN,NaN,NaN
4,4621892,Flesh & Blood TCG,Arcane Rising,Absorb in Aether (Blue),Rare,Damaged 1st Edition Normal,NaN,NaN,NaN


In [82]:
#naively, we can now compare buylist market/high to tcg market/low, and get cards that make money.
#we'll do this only on things that have values for all of that.
#TODO: change this to filters in case the number of tables changes
filtered = result.dropna(thresh=9).copy()

In [83]:
filtered.head()

,TCGplayer Id,Product Line,Set Name,Product Name,Rarity,Condition,TCG Market Price,TCG Low Price With Shipping,TCG Low Price
0,4621888,Flesh & Blood TCG,Arcane Rising,Absorb in Aether (Blue),Rare,Near Mint 1st Edition Normal,0.87,1.24,0.25
10,4680867,Flesh & Blood TCG,Arcane Rising,Absorb in Aether (Blue),Rare,Near Mint Unlimited Edition Normal,0.09,1.06,0.07
11,4680868,Flesh & Blood TCG,Arcane Rising,Absorb in Aether (Blue),Rare,Lightly Played Unlimited Edition Normal,0.06,1.01,0.02
12,4680869,Flesh & Blood TCG,Arcane Rising,Absorb in Aether (Blue),Rare,Moderately Played Unlimited Edition Normal,0.02,1.74,0.75
15,4680872,Flesh & Blood TCG,Arcane Rising,Absorb in Aether (Blue),Rare,Near Mint Unlimited Edition Rainbow Foil,0.27,1.14,0.15


In [84]:
filtered.shape

(11093, 9)

In [85]:
#creating new columns! All of this information is derived, so should maybe be put into functions?
#sellCut and buyCut are defined in cell 2

#we don't have a buylist to compare to, so we'll set some values in cell 2, and use those as estimates.

filtered['Optimistic-Market'] = filtered['TCG Market Price']*sellCut - filtered['TCG Market Price']*optimisticBuy
filtered['Pessimistic-Market'] = filtered['TCG Market Price']*sellCut - filtered['TCG Market Price']*pessimisticBuy
filtered['Optimistic-Low'] = filtered['TCG Low Price']*sellCut - filtered['TCG Low Price With Shipping']*optimisticBuy
filtered['Pessimistic-Low'] = filtered['TCG Low Price']*sellCut - filtered['TCG Low Price With Shipping']*pessimisticBuy

In [86]:
filtered.head()

,TCGplayer Id,Product Line,Set Name,Product Name,Rarity,Condition,TCG Market Price,TCG Low Price With Shipping,TCG Low Price,Optimistic-Market,Pessimistic-Market,Optimistic-Low,Pessimistic-Low
0,4621888,Flesh & Blood TCG,Arcane Rising,Absorb in Aether (Blue),Rare,Near Mint 1st Edition Normal,0.87,1.24,0.25,0.3306,0.0696,-0.4000,-0.7720
10,4680867,Flesh & Blood TCG,Arcane Rising,Absorb in Aether (Blue),Rare,Near Mint Unlimited Edition Normal,0.09,1.06,0.07,0.0342,0.0072,-0.4684,-0.7864
11,4680868,Flesh & Blood TCG,Arcane Rising,Absorb in Aether (Blue),Rare,Lightly Played Unlimited Edition Normal,0.06,1.01,0.02,0.0228,0.0048,-0.4874,-0.7904
12,4680869,Flesh & Blood TCG,Arcane Rising,Absorb in Aether (Blue),Rare,Moderately Played Unlimited Edition Normal,0.02,1.74,0.75,0.0076,0.0016,-0.2100,-0.7320
15,4680872,Flesh & Blood TCG,Arcane Rising,Absorb in Aether (Blue),Rare,Near Mint Unlimited Edition Rainbow Foil,0.27,1.14,0.15,0.1026,0.0216,-0.4380,-0.7800


In [87]:
#this is kind of a loaded name, but basically it's where the market makes sense. If the market price is lower than the lowest price, the market price is probably wrong.
#at the very least, something is weird, and we don't want to have a computer make a decision about weird things.
rational = filtered.query('`TCG Market Price` > `TCG Low Price With Shipping`')

In [88]:
rational.head()

,TCGplayer Id,Product Line,Set Name,Product Name,Rarity,Condition,TCG Market Price,TCG Low Price With Shipping,TCG Low Price,Optimistic-Market,Pessimistic-Market,Optimistic-Low,Pessimistic-Low
185,4621913,Flesh & Blood TCG,Arcane Rising,Aether Spindle (Red),Rare,Near Mint 1st Edition Rainbow Foil,14.58,13.99,13.00,5.5404,1.1664,4.4450,0.2480
205,4621933,Flesh & Blood TCG,Arcane Rising,Aether Spindle (Yellow),Rare,Near Mint 1st Edition Rainbow Foil,6.74,5.99,5.99,2.5612,0.5392,2.2762,0.4792
245,4621193,Flesh & Blood TCG,Arcane Rising,Amplify the Arknight (Red),Common,Near Mint 1st Edition Rainbow Foil,8.95,7.49,6.00,3.4010,0.7160,1.5350,-0.7120
291,4680907,Flesh & Blood TCG,Arcane Rising,Arcanite Skullcap,Legendary,Near Mint Unlimited Edition Rainbow Foil,38.61,35.98,34.99,14.6718,3.0888,12.8012,2.0072
292,4680908,Flesh & Blood TCG,Arcane Rising,Arcanite Skullcap,Legendary,Lightly Played Unlimited Edition Rainbow Foil,44.56,39.70,39.70,16.9328,3.5648,15.0860,3.1760


In [89]:
#How many cards behave the way do we expect, and what is the size of the data set we want to act on?
rational.shape

(1005, 13)

In [90]:
#And with that, the data is mostly there for you to peruse!

#Some examples!
#if optimistically, we're making less than $1, that's bad!
loss = rational.query('`Optimistic-Market` <= 1')
#if pessimistically, we're making more than $5, that's okay!
win = rational.query('`Pessimistic-Low` >= 5')

In [91]:
#How many wins do we have?
win.shape

(145, 13)

In [92]:
#What are the top wins?
win.sort_values('Pessimistic-Market')
win.head()

,TCGplayer Id,Product Line,Set Name,Product Name,Rarity,Condition,TCG Market Price,TCG Low Price With Shipping,TCG Low Price,Optimistic-Market,Pessimistic-Market,Optimistic-Low,Pessimistic-Low
331,4681217,Flesh & Blood TCG,Arcane Rising,Art of War,Majestic,Near Mint Unlimited Edition Rainbow Foil,102.67,87.78,87.00,39.0146,8.2136,32.6700,6.3360
332,4681218,Flesh & Blood TCG,Arcane Rising,Art of War,Majestic,Lightly Played Unlimited Edition Rainbow Foil,94.00,89.95,89.95,35.7200,7.5200,34.1810,7.1960
926,4622698,Flesh & Blood TCG,Arcane Rising,Command and Conquer,Majestic,Near Mint 1st Edition Normal,173.19,133.99,133.99,65.8122,13.8552,50.9162,10.7192
927,4622699,Flesh & Blood TCG,Arcane Rising,Command and Conquer,Majestic,Lightly Played 1st Edition Normal,179.99,179.00,179.00,68.3962,14.3992,68.0200,14.3200
941,4681207,Flesh & Blood TCG,Arcane Rising,Command and Conquer,Majestic,Near Mint Unlimited Edition Rainbow Foil,172.21,162.49,162.49,65.4398,13.7768,61.7462,12.9992


In [93]:
#How many losses?
loss.shape

(82, 13)

In [94]:
loss.sort_values('Optimistic-Market')
loss.head()

,TCGplayer Id,Product Line,Set Name,Product Name,Rarity,Condition,TCG Market Price,TCG Low Price With Shipping,TCG Low Price,Optimistic-Market,Pessimistic-Market,Optimistic-Low,Pessimistic-Low
336,4619383,Flesh & Blood TCG,Arcane Rising,"Azalea // Azalea, Ace in the Hole",Token,Near Mint 1st Edition Normal,2.10,1.69,0.70,0.7980,0.1680,-0.2290,-0.7360
2166,4681622,Flesh & Blood TCG,Arcane Rising,Lead the Charge (Red),Common,Near Mint Unlimited Edition Rainbow Foil,1.41,1.28,0.29,0.5358,0.1128,-0.3848,-0.7688
2607,4681178,Flesh & Blood TCG,Arcane Rising,Nullrune Robe,Common,Lightly Played Unlimited Edition Rainbow Foil,1.99,1.39,0.40,0.7562,0.1592,-0.3430,-0.7600
4141,4620478,Flesh & Blood TCG,Arcane Rising,Take Aim (Blue),Rare,Near Mint 1st Edition Normal,1.99,1.98,0.99,0.7562,0.1592,-0.1188,-0.7128
7493,7445866,Flesh & Blood TCG,Bright Lights,Evo Battery Pack,Common,Near Mint Cold Foil,2.16,1.98,0.99,0.8208,0.1728,-0.1188,-0.7128


In [95]:
#What if we look at all the things that make no sense?
strange = filtered.query('`TCG Market Price` < `TCG Low Price With Shipping`')
strange.sample(10)

,TCGplayer Id,Product Line,Set Name,Product Name,Rarity,Condition,TCG Market Price,TCG Low Price With Shipping,TCG Low Price,Optimistic-Market,Pessimistic-Market,Optimistic-Low,Pessimistic-Low
44535,4672100,Flesh & Blood TCG,Welcome to Rathe,Razor Reflex (Red),Common,Near Mint Unlimited Edition Rainbow Foil,2.15,2.85,1.86,0.8170,0.1720,0.2118,-0.6432
41119,4615012,Flesh & Blood TCG,Welcome Deck: Ira,Bittering Thorns,Promo,Near Mint 1st Edition Normal,0.31,1.24,0.25,0.1178,0.0248,-0.4000,-0.7720
757,4680118,Flesh & Blood TCG,Arcane Rising,Bull's Eye Bracers,Common,Lightly Played Unlimited Edition Normal,0.17,1.08,0.09,0.0646,0.0136,-0.4608,-0.7848
34500,5193712,Flesh & Blood TCG,Tales of Aria,Ball Lightning (Red),Common,Near Mint 1st Edition Rainbow Foil,0.46,1.14,0.15,0.1748,0.0368,-0.4380,-0.7800
35515,5308335,Flesh & Blood TCG,Tales of Aria,Earthlore Surge (Yellow),Common,Near Mint Unlimited Edition Normal,0.05,1.04,0.05,0.0190,0.0040,-0.4760,-0.7880
38010,5308380,Flesh & Blood TCG,Tales of Aria,Sow Tomorrow (Blue),Common,Near Mint Unlimited Edition Rainbow Foil,0.78,1.53,0.54,0.2964,0.0624,-0.2898,-0.7488
31135,4931029,Flesh & Blood TCG,Monarch,Wartune Herald (Yellow),Common,Lightly Played 1st Edition Normal,0.13,1.11,0.12,0.0494,0.0104,-0.4494,-0.7824
44675,4672230,Flesh & Blood TCG,Welcome to Rathe,Remembrance,Super Rare,Near Mint Unlimited Edition Rainbow Foil,12.57,18.44,18.44,4.7766,1.0056,7.0072,1.4752
8619,7438642,Flesh & Blood TCG,Bright Lights,Pulsewave Protocol,Majestic,Lightly Played,2.43,3.16,2.10,0.9234,0.1944,0.2680,-0.6800
27955,5020703,Flesh & Blood TCG,Monarch,Howl from Beyond (Yellow),Rare,Near Mint Unlimited Edition Normal,0.18,1.08,0.09,0.0684,0.0144,-0.4608,-0.7848


In [96]:
#Looking at all the strange cards in a set
strange.query('`Set Name` == "Dynasty"')

,TCGplayer Id,Product Line,Set Name,Product Name,Rarity,Condition,TCG Market Price,TCG Low Price With Shipping,TCG Low Price,Optimistic-Market,Pessimistic-Market,Optimistic-Low,Pessimistic-Low
15599,6722860,Flesh & Blood TCG,Dynasty,Aether Quickening (Blue),Rare,Near Mint,0.21,1.08,0.09,0.0798,0.0168,-0.4608,-0.7848
15600,6722861,Flesh & Blood TCG,Dynasty,Aether Quickening (Blue),Rare,Lightly Played,0.25,1.09,0.10,0.0950,0.0200,-0.4570,-0.7840
15604,6722865,Flesh & Blood TCG,Dynasty,Aether Quickening (Blue),Rare,Near Mint Rainbow Foil,0.49,1.08,0.09,0.1862,0.0392,-0.4608,-0.7848
15605,6722866,Flesh & Blood TCG,Dynasty,Aether Quickening (Blue),Rare,Lightly Played Rainbow Foil,0.85,1.48,0.49,0.3230,0.0680,-0.3088,-0.7528
15609,6722840,Flesh & Blood TCG,Dynasty,Aether Quickening (Red),Rare,Near Mint,0.16,1.04,0.05,0.0608,0.0128,-0.4760,-0.7880
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18242,6722205,Flesh & Blood TCG,Dynasty,Withstand (Yellow),Common,Near Mint Rainbow Foil,0.16,1.09,0.10,0.0608,0.0128,-0.4570,-0.7840
18247,6721455,Flesh & Blood TCG,Dynasty,"Yoji, Royal Protector",Majestic,Near Mint,0.44,1.19,0.20,0.1672,0.0352,-0.4190,-0.7760
18248,6721456,Flesh & Blood TCG,Dynasty,"Yoji, Royal Protector",Majestic,Lightly Played,0.40,1.34,0.35,0.1520,0.0320,-0.3620,-0.7640
18252,6721460,Flesh & Blood TCG,Dynasty,"Yoji, Royal Protector",Majestic,Near Mint Rainbow Foil,0.93,1.73,0.74,0.3534,0.0744,-0.2138,-0.7328


In [97]:
#how many things are in our query?
strange.query('`Set Name` == "Dynasty"').shape

(717, 13)

In [98]:
#What are the cards where the difference is more than 30%, and the market price is more than $1?
strange.query('(abs((`TCG Market Price` / `TCG Low Price With Shipping`) - 1) > .30) and `TCG Market Price` > 1' )

,TCGplayer Id,Product Line,Set Name,Product Name,Rarity,Condition,TCG Market Price,TCG Low Price With Shipping,TCG Low Price,Optimistic-Market,Pessimistic-Market,Optimistic-Low,Pessimistic-Low
75,4679777,Flesh & Blood TCG,Arcane Rising,Achilles Accelerator,Common,Near Mint Unlimited Edition Rainbow Foil,1.07,4.99,4.00,0.4066,0.0856,1.0250,-0.4720
145,4619803,Flesh & Blood TCG,Arcane Rising,Aether Sink,Rare,Near Mint 1st Edition Rainbow Foil,9.50,15.00,15.00,3.6100,0.7600,5.7000,1.2000
195,4680882,Flesh & Blood TCG,Arcane Rising,Aether Spindle (Red),Rare,Near Mint Unlimited Edition Rainbow Foil,2.14,3.13,2.14,0.8132,0.1712,0.3182,-0.6208
196,4680883,Flesh & Blood TCG,Arcane Rising,Aether Spindle (Red),Rare,Lightly Played Unlimited Edition Rainbow Foil,2.48,4.09,2.10,0.9424,0.1984,-0.1970,-1.4240
265,4621218,Flesh & Blood TCG,Arcane Rising,Amplify the Arknight (Yellow),Common,Near Mint 1st Edition Rainbow Foil,2.28,7.10,7.10,0.8664,0.1824,2.6980,0.5680
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45965,4616937,Flesh & Blood TCG,Welcome to Rathe,Warrior's Valor (Blue),Rare,Near Mint 1st Edition Rainbow Foil,34.98,54.94,49.95,13.2924,2.7984,16.4860,0.0040
46005,4616917,Flesh & Blood TCG,Welcome to Rathe,Warrior's Valor (Yellow),Rare,Near Mint 1st Edition Rainbow Foil,14.21,79.95,79.95,5.3998,1.1368,30.3810,6.3960
46046,4616727,Flesh & Blood TCG,Welcome to Rathe,Whelming Gustwave (Red),Common,Near Mint 1st Edition Normal,2.03,3.26,2.27,0.7714,0.1624,0.3676,-0.6104
46091,4618062,Flesh & Blood TCG,Welcome to Rathe,Wounded Bull (Blue),Common,Near Mint 1st Edition Rainbow Foil,3.45,4.97,3.98,1.3110,0.2760,1.0174,-0.4736


In [101]:
#Among cards that make sense, mid-range LP Foils!
rational.query('(50 > `TCG Market Price` > 10) and `Condition` == "Lightly Played Unlimited Edition Rainbow Foil"').sort_values('Pessimistic-Low').tail()

,TCGplayer Id,Product Line,Set Name,Product Name,Rarity,Condition,TCG Market Price,TCG Low Price With Shipping,TCG Low Price,Optimistic-Market,Pessimistic-Market,Optimistic-Low,Pessimistic-Low
45666,4671731,Flesh & Blood TCG,Welcome to Rathe,Steelblade Supremacy,Majestic,Lightly Played Unlimited Edition Rainbow Foil,24.98,13.75,13.75,9.4924,1.9984,5.2250,1.1000
10001,5141390,Flesh & Blood TCG,Crucible of War,Breeze Rider Boots,Majestic,Lightly Played Unlimited Edition Rainbow Foil,17.36,16.02,16.02,6.5968,1.3888,6.0876,1.2816
3867,4680813,Flesh & Blood TCG,Arcane Rising,Sonic Boom,Majestic,Lightly Played Unlimited Edition Rainbow Foil,19.87,18.05,18.05,7.5506,1.5896,6.8590,1.4440
662,4680803,Flesh & Blood TCG,Arcane Rising,Blazing Aether,Majestic,Lightly Played Unlimited Edition Rainbow Foil,19.16,18.15,18.15,7.2808,1.5328,6.8970,1.4520
36541,5308396,Flesh & Blood TCG,Tales of Aria,Heart of Ice,Legendary,Lightly Played Unlimited Edition Rainbow Foil,35.74,21.20,21.20,13.5812,2.8592,8.0560,1.6960
